In [2]:
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('data/istanbul_data_fully_preprocessed.csv')

data.info()

name_cols = ['MainDistID', 'DistID', 'HcoID', 'SkuID', 'BrandID', 'District']
categorical_cols = ['HcoType', 'SalesRegionID'] 
numerical_cols = ['PaidQty', 'FGQty', 'TotalQuantity', 'Week', 'Quarter', 'Month', 'Year']
boolean_cols = ['IsReturn', 'IsMrsOrder', 'Status']

# convert categorical columns to 'category' data type
for col in categorical_cols:
    data[col] = data[col].astype('category')

# convert numerical columns to 'int' data type
for col in numerical_cols:
    data[col] = data[col].astype('int')
    
# convert name columns to str data type
for col in name_cols:
    data[col] = data[col].astype(str)

# convert boolean columns to 'bool' data type
for col in boolean_cols:
    data[col] = data[col].astype(bool)
    
raw_data = data.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 863002 entries, 0 to 863001
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   MainDistID     863002 non-null  int64 
 1   DistID         863002 non-null  int64 
 2   Date           863002 non-null  object
 3   HcoID          863002 non-null  int64 
 4   BrandID        863002 non-null  int64 
 5   SkuID          863002 non-null  int64 
 6   IsReturn       863002 non-null  bool  
 7   PaidQty        863002 non-null  int64 
 8   FGQty          863002 non-null  int64 
 9   IsMrsOrder     863002 non-null  bool  
 10  HcoType        863002 non-null  object
 11  District       857497 non-null  object
 12  Status         863002 non-null  bool  
 13  TotalQuantity  863002 non-null  int64 
 14  Week           863002 non-null  int64 
 15  Month          863002 non-null  int64 
 16  Quarter        863002 non-null  int64 
 17  Year           863002 non-null  int64 
 18  Sale

In [3]:
segment_1_sku_ids = ["568", "578", "1050"]
segment_2_sku_ids = ["319", "509", "615", "1018"]

all_sku_ids = segment_1_sku_ids + segment_2_sku_ids

data = data[data['SkuID'].isin(all_sku_ids)]

In [4]:
quarterly_data = data.copy()

quarterly_data = quarterly_data.groupby(['Year', 'Quarter', 'SkuID']).agg({'TotalQuantity': 'sum'}).reset_index()
quarterly_data

,Year,Quarter,SkuID,TotalQuantity
0,2019,1,1018,13419
1,2019,1,1050,50137
2,2019,1,319,44549
3,2019,1,509,41402
4,2019,1,568,138149
...,...,...,...,...
79,2021,4,319,76364
80,2021,4,509,36845
81,2021,4,568,225881
82,2021,4,578,109923


In [5]:
sku_319 = quarterly_data[quarterly_data['SkuID'] == "319"]
sku_319

,Year,Quarter,SkuID,TotalQuantity
2,2019,1,319,44549
9,2019,2,319,40391
16,2019,3,319,40588
23,2019,4,319,47633
30,2020,1,319,55259
37,2020,2,319,48960
44,2020,3,319,50755
51,2020,4,319,59189
58,2021,1,319,62148
65,2021,2,319,53325


In [6]:
import itertools
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

# Parametre kombinasyonlarını oluştur
p_values = range(0, 3) # Örnek olarak 0'dan 3'e kadar p değerleri
d_values = range(0, 2) # Örnek olarak 0'dan 2'ye kadar d değerleri
q_values = range(0, 3) # Örnek olarak 0'dan 3'e kadar q değerleri
param_combinations = list(itertools.product(p_values, d_values, q_values))

best_score, best_params, best_model, mape = float("inf"), None, None, None

df = sku_319.copy()
df['Date'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Quarter'].astype(str))
df.set_index('Date', inplace=True)
df.index = df.index.to_period('Q').to_timestamp('S')

# Grid search
for param in param_combinations:
    try:
        model = ARIMA(df['TotalQuantity'], order=param)
        model_fit = model.fit()
        y_pred = model_fit.forecast(steps=2)  
        mse = mean_squared_error(df['TotalQuantity'].iloc[-2:], y_pred)
        mape = mse / df['TotalQuantity'].iloc[-2:].mean()
        if mse < best_score:
            best_score, best_params, best_model, mape = mse, param, model_fit, mape
    except:
        continue

/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `sta

# Best Parameters for ARIMA Model

In [7]:
print('Best ARIMA Model:', best_params)
print('MSE:', best_score)
print('MAPE:', mape)
print(best_model.summary())

Best ARIMA Model: (1, 0, 0)
MSE: 227759897.1466329
MAPE: 4335.985923515653
                               SARIMAX Results                                
Dep. Variable:          TotalQuantity   No. Observations:                   12
Model:                 ARIMA(1, 0, 0)   Log Likelihood                -125.899
Date:                Sun, 02 Jun 2024   AIC                            257.797
Time:                        15:58:15   BIC                            259.252
Sample:                             0   HQIC                           257.259
                                 - 12                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        5.26e+04   5615.828      9.367      0.000    4.16e+04    6.36e+04
ar.L1          0.5665      0.659      0.860      0.390  

In [8]:
import itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

dfSarimax = sku_319.copy()
df['Date'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Quarter'].astype(str))
df.set_index('Date', inplace=True)

# Parametre kombinasyonlarını oluştur
p_values = range(0, 3) # Örnek olarak 0'dan 3'e kadar p değerleri
d_values = range(0, 2) # Örnek olarak 0'dan 2'ye kadar d değerleri
q_values = range(0, 3) # Örnek olarak 0'dan 3'e kadar q değerleri
P_values = range(0, 3) # Örnek olarak 0'dan 3'e kadar P değerleri
D_values = range(0, 2) # Örnek olarak 0'dan 2'ye kadar D değerleri
Q_values = range(0, 3) # Örnek olarak 0'dan 3'e kadar Q değerleri
s_values = range(4, 5) # Örnek olarak 4'den 5'e kadar s değerleri
param_combinations = list(itertools.product(p_values, d_values, q_values, P_values, D_values, Q_values, s_values))

best_score, best_params, best_model, mape = float("inf"), None, None, None

# Grid search
for param in param_combinations:
    try:
        model = SARIMAX(dfSarimax['TotalQuantity'], order=param[:3], seasonal_order=param[3:])
        model_fit = model.fit()
        y_pred = model_fit.forecast(steps=2)
        mse = mean_squared_error(dfSarimax['TotalQuantity'].iloc[-2:], y_pred)
        mape = mse / dfSarimax['TotalQuantity'].iloc[-2:].mean()
        if mse < best_score:
            best_score, best_params, best_model, mape = mse, param, model_fit, mape
    except:
        continue

/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
 This problem is unconstrained.
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/ts

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            1     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.23059D+01    |proj g|=  3.55271D-10

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    1      0      1      0     0     0   3.553D-10   1.231D+01
  F =   12.305886856040592     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            2     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.6

 This problem is unconstrained.



At iterate   10    f=  1.25403D+01    |proj g|=  1.51156D-02

At iterate   15    f=  1.24813D+01    |proj g|=  7.43843D-04

At iterate   20    f=  1.22862D+01    |proj g|=  2.16665D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     24     25      1     0     0   7.422D-06   1.181D+01
  F =   11.809963713585036     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            1     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  7.11280D+00    |proj g|=  4.11260D-05

       

/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
 This problem is unconstrained.
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/senaoz/anaconda3/lib/python3.11/site-

...

/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
 This problem is unconstrained.



At iterate   15    f=  5.84231D+00    |proj g|=  1.21286D-02

At iterate   20    f=  5.84177D+00    |proj g|=  3.20261D-03

At iterate   25    f=  5.84172D+00    |proj g|=  3.06909D-02

At iterate   30    f=  5.83876D+00    |proj g|=  3.50832D-01

At iterate   35    f=  5.81155D+00    |proj g|=  5.27902D-01

At iterate   40    f=  5.74379D+00    |proj g|=  1.41088D-01

At iterate   45    f=  5.73321D+00    |proj g|=  1.50948D-01

At iterate   50    f=  5.73290D+00    |proj g|=  6.12340D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    8     50     70      1     0     0   6.123D-03   5.733D+00
  F =   5.73289666113

/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            9     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.95060D+00    |proj g|=  4.10296D-01

At iterate    5    f=  5.86493D+00    |proj g|=  2.29694D-01

At iterate   10    f=  5.84174D+00    |proj g|=  1.94326D-02

At iterate   15    f=  5.83849D+00    |proj g|=  1.40497D-02

At iterate   20    f=  5.83838D+00    |proj g|=  6.77979D-03

At iterate   25    f=  5.83803D+00    |proj g|=  7.93150D-03

At iterate   30    f=  5.83792D+00    |proj g|=  1.31853D-02

At iterate   35    f=  5.83593D+00    |proj g|=  9.95746D-02

At iterate   40    f=  5.82604D+00    |proj g|=  2.37497D-01

At iterate   45    f=  5.78235D+00    |proj g|=  3.73266D-02

At iterate   50    f=  5.76940D+00    |proj g|=  7.26797D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cau

/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/senaoz/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


# Best Parameters for SARIMA Model

In [10]:
print('Best SARIMA Model:', best_params)
print('MSE:', best_score)
print('MAPE:', mape)
print(best_model.summary())

Best SARIMA Model: (2, 1, 2, 2, 0, 1, 4)
MSE: 135384522.39222947
MAPE: 6554.235724834278
                                      SARIMAX Results                                      
Dep. Variable:                       TotalQuantity   No. Observations:                   12
Model:             SARIMAX(2, 1, 2)x(2, 0, [1], 4)   Log Likelihood                -113.639
Date:                             Sun, 02 Jun 2024   AIC                            243.277
Time:                                     15:59:21   BIC                            246.460
Sample:                                          0   HQIC                           241.271
                                              - 12                                         
Covariance Type:                               opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -1.